In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

This is the notebook version (used for testing purposes) of the scripts create_weather_related_features.py and create_weather_related_features_parallelized.py.
* Use create_weather_related_features_parallelized.py (fastest)

In [8]:
# import subset master dataset data
subset_df = pd.read_pickle('./../data/subset_master_dataset.pkl')
print(subset_df.shape)
display(subset_df.head())
display(subset_df.info())

(5565, 13)


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,true_SOD_BP
1992,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.07.JPG,00400124.07.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,torso
1995,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.10.JPG,00400124.10.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,legs
1996,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.11.JPG,00400124.11.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,legs
1997,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.12.JPG,00400124.12.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,head
1998,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.13.JPG,00400124.13.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,head


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 1992 to 1129561
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              5565 non-null   object        
 1   old_id              5565 non-null   object        
 2   img_path            5565 non-null   object        
 3   img                 5565 non-null   object        
 4   correct_img_date    5565 non-null   datetime64[ns]
 5   date_placed_ARF     5565 non-null   datetime64[ns]
 6   PMI_days            5565 non-null   float64       
 7   sex                 5565 non-null   float64       
 8   age_at_death        5565 non-null   float64       
 9   est_stature_inches  5565 non-null   float64       
 10  est_weight          5565 non-null   float64       
 11  true_SOD            5565 non-null   float64       
 12  true_SOD_BP         5565 non-null   object        
dtypes: datetime64[ns](2), float64(6), object(5

None

In [9]:
subset_df.true_SOD_BP.unique()

array(['torso', 'legs', 'head', 'arm'], dtype=object)

In [10]:
# import LCD weather data (hourly)
weather_df = pd.read_pickle('/data/anau/temp_humidity_data/data/LCD/lcd_hourly.pkl')
print(weather_df.shape)
display(weather_df.head())
display(weather_df.info())

(107340, 3)


,date_time,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2011-01-01 00:00:00,12.033333,90.333333
1,2011-01-01 01:00:00,10.575000,92.250000
2,2011-01-01 02:00:00,10.600000,92.000000
3,2011-01-01 03:00:00,10.425000,92.000000
4,2011-01-01 04:00:00,10.340000,92.800000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 107340 entries, 0 to 107339
Data columns (total 3 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date_time                 107340 non-null  datetime64[ns]
 1   HourlyDryBulbTemperature  107340 non-null  float64       
 2   HourlyRelativeHumidity    107340 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 3.3 MB


None

In [ ]:
subset_dict = subset_df.to_dict('records')
row_counter = 0
# for each image
for i,row in enumerate(subset_dict):
    print('iter:', i, 'ID:', row['new_id'])
    placement_date = row['date_placed_ARF']
    img_date = row['correct_img_date']

    ADD_thres0 = 0
    ADD_thres5 = 0
    ADD_thres10 = 0
    ADD_thres15 = 0
    
    ADH_thres0 = 0
    ADH_thres5 = 0
    ADH_thres10 = 0
    ADH_thres15 = 0
    while (placement_date <= img_date):
        print('   ', placement_date, img_date)
        # get weather for placement_date
        weather_current_df = weather_df[weather_df.date_time.dt.date == placement_date]
        #print(weather_current_df)

        # calculate ADD with varying temp thresholds
        avg_temp = weather_current_df.loc[:, 'HourlyDryBulbTemperature'].mean()
        print('     ', 'daily_avg_temp:', avg_temp)
        if avg_temp >= 0:
            ADD_thres0 += avg_temp
        if avg_temp >= 5:
            ADD_thres5 += avg_temp
        if avg_temp >= 10:
            ADD_thres10 += avg_temp
        if avg_temp >= 15:
            ADD_thres15 += avg_temp
        

        # calculate ADH with varying temp thresholds
        total_temp_thres0 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 0]['HourlyDryBulbTemperature'].sum()
        ADH_thres0 += total_temp_thres0
        print('     ', 'daily_total_temp_thres0:', total_temp_thres0)
        
        total_temp_thres5 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 5]['HourlyDryBulbTemperature'].sum()
        ADH_thres5 += total_temp_thres5
        print('     ', 'daily_total_temp_thres5:', total_temp_thres5)
        
        total_temp_thres10 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 10]['HourlyDryBulbTemperature'].sum()
        ADH_thres10 += total_temp_thres10
        print('     ', 'daily_total_temp_thres10:', total_temp_thres10)
        
        total_temp_thres15 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 15]['HourlyDryBulbTemperature'].sum()
        ADH_thres15 += total_temp_thres15
        print('     ', 'daily_total_temp_thres15:', total_temp_thres15)

        # go to the next day
        placement_date += datetime.timedelta(days=1)

    # add to dict
    row['ADD_thres0'] = ADD_thres0
    row['ADD_thres5'] = ADD_thres5
    row['ADD_thres10'] = ADD_thres10
    row['ADD_thres15'] = ADD_thres15
    
    row['ADH_thres0'] = ADH_thres0
    row['ADH_thres5'] = ADH_thres5
    row['ADH_thres10'] = ADH_thres10
    row['ADH_thres15'] = ADH_thres15
    print(row)

    print()
    print()

# convert dict to dataframe
#subset_w_weather_df = pd.DataFrame.from_dict(subset_dict)
# write to pickle
#subset_w_weather_df.to_pickle('./../data/subset_master_dataset_w_weather.pkl')

## After running create_weather_related_features_parallelized.py

In [2]:
with open('./../data/subset_master_dataset_w_weather.pkl', 'rb') as f:
        data_ls = pickle.load(f)
print(len(data_ls))

5565


In [3]:
# convert list of dicts to dataframe
data_df = pd.DataFrame.from_dict(data_ls)
print(data_df.shape)
display(data_df.head())
display(data_df.info())
display(data_df.describe())

(5565, 21)


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,true_SOD_BP,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
0,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.07.JPG,00400124.07.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,torso,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
1,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.10.JPG,00400124.10.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,legs,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
2,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.11.JPG,00400124.11.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,legs,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
3,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.12.JPG,00400124.12.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,head,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
4,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.13.JPG,00400124.13.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,180.0,3.0,head,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              5565 non-null   object        
 1   old_id              5565 non-null   object        
 2   img_path            5565 non-null   object        
 3   img                 5565 non-null   object        
 4   correct_img_date    5565 non-null   datetime64[ns]
 5   date_placed_ARF     5565 non-null   datetime64[ns]
 6   PMI_days            5565 non-null   float64       
 7   sex                 5565 non-null   float64       
 8   age_at_death        5565 non-null   float64       
 9   est_stature_inches  5565 non-null   float64       
 10  est_weight          5565 non-null   float64       
 11  true_SOD            5565 non-null   float64       
 12  true_SOD_BP         5565 non-null   object        
 13  ADD_thres0          5565 non-null   float64     

None

,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
count,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000
mean,49.664331,0.484816,66.056604,155.004972,184.267565,3.527763,708.290345,690.390777,635.252086,526.452252,17123.411344,16750.067399,15539.655443,13146.278034
std,55.519650,0.499814,14.540983,38.573655,58.632396,1.157832,897.411421,889.248872,863.837315,788.885445,21600.033467,21403.362741,20699.314494,19118.359568
min,0.000000,0.000000,21.000000,0.000000,95.000000,1.000000,0.735417,0.000000,0.000000,0.000000,55.500000,28.450000,0.000000,0.000000
25%,14.000000,0.000000,58.000000,160.020000,144.000000,3.000000,169.334067,153.600799,115.196181,53.183333,4356.442619,3890.429167,3155.655000,1854.531667
50%,31.000000,0.000000,67.000000,167.640000,174.000000,3.000000,409.313144,386.366424,342.317959,228.923148,9914.536190,9472.987500,8609.778333,6151.056667
75%,71.000000,1.000000,76.000000,175.260000,210.000000,4.000000,902.689834,876.523317,834.103166,622.933814,21615.516667,21425.544008,19793.244008,15544.623333
max,450.000000,1.000000,96.000000,195.580000,516.000000,6.000000,5807.064398,5619.145350,5229.522822,4511.635084,140976.538945,137072.899421,126508.944401,110144.846306


In [48]:
data_df.sample(1)

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,true_SOD_BP,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
608,082,UT06-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/082/08200224.24.JPG,08200224.24.JPG,2013-02-24,2013-01-23,32.0,1.0,59.0,182.88,220.0,3.0,torso,174.904965,153.600799,37.778611,0.0,4440.059167,3890.429167,1969.445,405.45
